In [ ]:
import pandas as pd

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
df = pd.read_pickle('issues_rnd_sample.pkl')

In [ ]:
df_t = pd.read_pickle('issues_refined_12_04_21_12_27.pkl')

In [ ]:
df_t.shape

(29014, 9)

In [ ]:
df_t.head()

,url,id,number,title,labels,state,created_at,closed_at,body
0,https://api.github.com/repos/johnmichel/Librar...,795439989,189,"[jquery, version, detection, appears, incorrect]",,closed,2021-01-27T21:00:24Z,2021-01-27T21:24:19Z,"[test, url, went, every, j, loaded, page, far,..."
3,https://api.github.com/repos/johnmichel/Librar...,723259346,186,"[fix, angularjs, detection, false, positive]",,open,2020-10-16T14:07:20Z,None,"[angularjs, beginning, versioning, object, rem..."
11,https://api.github.com/repos/johnmichel/Librar...,717018583,178,"[error, workbox, library, block]",,closed,2020-10-08T04:51:41Z,2021-01-06T14:25:24Z,"[running, extension, master, error, workbox, b..."
12,https://api.github.com/repos/johnmichel/Librar...,696283905,177,"[updated, script, compatible, firefox]",,open,2020-09-09T00:40:59Z,None,"[try, chrome, change, suppose, work, correctly..."
16,https://api.github.com/repos/johnmichel/Librar...,654133470,173,"[improve, preact, detection, performance]",,closed,2020-07-09T15:04:13Z,2020-07-09T21:57:41Z,"[fast, path, test, preact, detecting, preact, ..."


In [ ]:
keys = 'access policy, access role, access-policy, access-role, accesspolicy,  accessrole,  aes,  audit,  authentic,  authority,  authoriz,  biometric,  black  list,  black-list,  blacklist,  blacklist,cbc, certificate, checksum, cipher, clearance, confidentiality,cookie,  crc,  credential,  crypt,  csrf,  decode,  defensive  programming, defensive-programming, delegation, denial of service, denial-of-service, diffie-hellman, dmz, dotfuscator, dsa,ecdsa,  encode,  escrow,  exploit,  firewall,  forge,  forgery,  gssapi,  gss-api,  gssapi,  hack,  hash,  hmac,  honey  pot,  honey-pot,  honeypot,  inject,  integrity,  kerberos,  ldap,  login,  mal-ware,  md5,  nonce,  nss,  oauth,  obfuscat,  open  auth,  open-auth, openauth, openid, owasp, password, pbkdf2, pgp, phishing, pki, privacy, private key, private-key, privatekey, privilege, public key, public-key, publickey, rbac, rc4, repudiation,rfc 2898, rfc-2898, rfc2898, rijndael, rootkit, rsa, salt, saml,sanitiz,  secur,  sha,  shell  code,  shell-code,  shellcode,  shibboleth,  signature,  signed,  signing,  sing  sign-on,  single  signon,  single-sign-on,  smart  assembly,  smart-assembly,  smartassembly,  snif,  spam,  spnego,  spoofing,  spyware,  ssl,  sso,steganography,  tampering,  trojan,  trust,  violat,  virus,  whitelist, white-list, whitelist, x509, xss'

In [ ]:
keys = keys.split(',')

In [ ]:
keys = [k.strip() for k in keys]

In [ ]:
df_t['title_stj'] = df_t['title'].apply(lambda x: ' '.join([ps.stem(w) for w in x]))

In [ ]:
df_t['body_stj'] = df_t['body'].apply(lambda x: ' '.join([ps.stem(w) for w in x]))

In [ ]:
df_t.head()

,url,id,number,title,labels,state,created_at,closed_at,body,title_stj,body_stj
0,https://api.github.com/repos/johnmichel/Librar...,795439989,189,"[jquery, version, detection, appears, incorrect]",,closed,2021-01-27T21:00:24Z,2021-01-27T21:24:19Z,"[test, url, went, every, j, loaded, page, far,...",jqueri version detect appear incorrect,test url went everi j load page far tell jquer...
3,https://api.github.com/repos/johnmichel/Librar...,723259346,186,"[fix, angularjs, detection, false, positive]",,open,2020-10-16T14:07:20Z,None,"[angularjs, beginning, versioning, object, rem...",fix angularj detect fals posit,angularj begin version object remov els condit...
11,https://api.github.com/repos/johnmichel/Librar...,717018583,178,"[error, workbox, library, block]",,closed,2020-10-08T04:51:41Z,2021-01-06T14:25:24Z,"[running, extension, master, error, workbox, b...",error workbox librari block,run extens master error workbox block thrown j...
12,https://api.github.com/repos/johnmichel/Librar...,696283905,177,"[updated, script, compatible, firefox]",,open,2020-09-09T00:40:59Z,None,"[try, chrome, change, suppose, work, correctly...",updat script compat firefox,tri chrome chang suppos work correctli thank g...
16,https://api.github.com/repos/johnmichel/Librar...,654133470,173,"[improve, preact, detection, performance]",,closed,2020-07-09T15:04:13Z,2020-07-09T21:57:41Z,"[fast, path, test, preact, detecting, preact, ...",improv preact detect perform,fast path test preact detect preact x site use...


In [ ]:
k = df_t.title_stj.apply(lambda x: any(k in x for k in keys))
l = df_t.body_stj.apply(lambda x: any(k in x for k in keys))

In [ ]:
from collections import Counter

In [ ]:
Counter(l), Counter(k)

(Counter({False: 26635, True: 2379}), Counter({False: 28159, True: 855}))

In [ ]:
df_t_title = df_t.loc[k]

In [ ]:
df_t_body = df_t.loc[l]

In [ ]:
df_t_security = pd.concat([df_t_title, df_t_body])

In [ ]:
df_t_security.shape, df_t.shape

((2705, 10), (29014, 11))

In [ ]:
df_t_security.drop_duplicates(['id'], inplace=True)

In [ ]:
df_t_security.head()

,url,id,number,title,labels,state,created_at,closed_at,body,title_stj,body_stj
170,https://api.github.com/repos/johnmichel/Librar...,8312690,19,"[x, domain, error, injected, script]",,closed,2012-11-13T04:55:29Z,2014-06-06T03:45:51Z,"[appeared, updating, manifest, file, need, spe...",x domain error inject script,appear updat manifest file need spend time rev...
1067,https://api.github.com/repos/OctoLinker/OctoLi...,349844054,494,"[get, parent, sha, pr, diff, view]",,closed,2018-08-12T21:06:55Z,2018-08-13T20:39:37Z,"[journey, get, pull, request, working, beginni...",get parent sha pr diff view,journey get pull request work begin discov sel...
1265,https://api.github.com/repos/OctoLinker/OctoLi...,210397886,293,"[ignre, checksum, installing, chrome, choco]",,closed,2017-02-27T06:25:11Z,2017-02-27T06:28:55Z,"[downloading, googlechrome, bit, progress, com...",ignr checksum instal chrome choco,download googlechrom bit progress complet down...
1588,https://api.github.com/repos/jordwest/news-fee...,706467182,97,"[support, eradication, hacker, news, news, yco...",,closed,2020-09-22T15:07:39Z,2020-10-01T08:14:02Z,[],support erad hacker news news ycombin com,
1594,https://api.github.com/repos/jordwest/news-fee...,672394513,91,"[whitelist, blacklist, certain, page, people]",,closed,2020-08-03T23:10:18Z,2020-08-05T00:33:18Z,"[use, case, sometimes, certain, update, nice, ...",whitelist blacklist certain page peopl,use case sometim certain updat nice receiv loc...


In [ ]:
df_t_security.drop(['title_stj', 'body_stj'], axis=1, inplace=True)

In [ ]:
df_t_security.to_pickle('security_issues_15_05_2020.pkl')

In [ ]:
df_t_security.columns

Index(['url', 'id', 'number', 'title', 'labels', 'state', 'created_at',
       'closed_at', 'body'],
      dtype='object')

In [ ]:
df_t_security['year'] = df_t_security['created_at'].apply(lambda x: str(x).split('-')[0])

In [ ]:
sec_issues_by_year = {}

In [ ]:
issues_by_year = {}

In [ ]:
for t in df_t_security.iterrows():
  if t[1]['year'] not in sec_issues_by_year:
    sec_issues_by_year[t[1]['year']] = 1
  else:
    sec_issues_by_year[t[1]['year']] += 1
  # break

In [ ]:
for t in df_t.iterrows():
  year = str(t[1]['created_at']).split('-')[0]
  if year not in issues_by_year:
    issues_by_year[year] = 1
  else:
    issues_by_year[year] += 1
  # break

In [ ]:
sec_issues_by_year

{'2010': 0,
 '2011': 11,
 '2012': 43,
 '2013': 62,
 '2014': 65,
 '2015': 82,
 '2016': 223,
 '2017': 405,
 '2018': 416,
 '2019': 546,
 '2020': 686,
 '2021': 166}

In [ ]:
issues_by_year

{'2010': 21,
 '2011': 47,
 '2012': 546,
 '2013': 593,
 '2014': 843,
 '2015': 892,
 '2016': 2562,
 '2017': 3660,
 '2018': 4418,
 '2019': 5641,
 '2020': 7799,
 '2021': 1992}

In [ ]:
res = []

In [ ]:
for i in issues_by_year:
  res.append({
      'year': i,
      'issues': issues_by_year[i],
      'security_issues': sec_issues_by_year[i]
  })

In [ ]:
pd.DataFrame(res).sort_values('year').to_csv('security_issues_stats.csv')

In [ ]:
n = Counter(df_t['url'].apply(lambda x: str(x).split('/')[-3]))

In [ ]:
count = 0
for i in n:
  count += 1

In [ ]:
count

241